In [38]:
import os
import sys
import numpy as np
import scipy as sp
import scipy.stats as stats
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
from scipy.stats import shapiro
import pandas as pd
import sklearn as sk
import virtualenv
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import pysqlite3
import sqlite3
import pymysql as ps


In [39]:
print("version of python")
print(sys.version)

version of python
3.9.13 (main, Oct 13 2022, 16:12:30) 
[Clang 12.0.0 ]


In [41]:
df = pd.read_csv("cell-count_copy.csv")
print(df.head(2))
print(df.columns)

  project subject condition  age sex treatment response sample sample_type  \
0    prj1    sbj1  melanoma   70   F       tr1        y     s1        PBMC   
1    prj1    sbj1  melanoma   70   F       tr1        y     s2        PBMC   

   time_from_treatment_start  b_cell  cd8_t_cell  cd4_t_cell  nk_cell  \
0                        0.0   36000       24000       42000     6000   
1                        7.0   30000       22000       40000     2000   

   monocyte  
0     12000  
1      6000  
Index(['project', 'subject', 'condition', 'age', 'sex', 'treatment',
       'response', 'sample', 'sample_type', 'time_from_treatment_start',
       'b_cell', 'cd8_t_cell', 'cd4_t_cell', 'nk_cell', 'monocyte'],
      dtype='object')


In [42]:
# To design a simple database, we can use the package sqlite3 

In [43]:
# In the code below, we perform the following operations : 
# we create a new SQLite database that is named "cell_counts.db".
# A table named "cell_counts" is created with columns corresponding to the DataFrame columns.
# The data from the DataFrame is inserted into the SQLite table.
# The data is then queried back from the database and displayed.

In [44]:
# How would you design a database to capture the type of information and data in cell-count.csv? 
# Imagine that you’d have hundreds of projects, thousands of samples and various types of analytics you’d want to perform, 
# including the example analysis of responders versus non-responders comparisons above. Please provide a rough prototype schema.

In [45]:
# Connect to SQLite database ; let's call it "TEIKO CELL COUNTS"
conn = sqlite3.connect('cell_counts.db')

# Create a cursor object
cursor = conn.cursor()

# Create a table
cursor.execute('''
    CREATE TABLE cell_counts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        project TEXT, 
        subject TEXT,
        condition  TEXT, 
        age  TEXT,
        sex  TEXT,
        treatment  TEXT,
        reponse  TEXT,
        sample  TEXT,
        sample_type  TEXT,
        time_from_treatment_start  TEXT,
        b_cell INTEGER,
        cd8_t_cell INTEGER,
        cd4_t_cell INTEGER,
        nk_cell INTEGER,
        monocyte INTEGER
    )
''')

# "to_sql" : Insert DataFrame data into the TABLE
df.to_sql('cell_counts', conn, index=False, if_exists='replace')

# Commit the changes
conn.commit()

# Query data from the database
query_result = pd.read_sql('SELECT * FROM cell_counts', conn)

# Display the result
print(query_result)

# Close the connection
# conn.close()

   project subject condition  age sex treatment response sample sample_type  \
0     prj1    sbj1  melanoma   70   F       tr1        y     s1        PBMC   
1     prj1    sbj1  melanoma   70   F       tr1        y     s2        PBMC   
2     prj1    sbj1  melanoma   70   F       tr1        y     s3        PBMC   
3     prj1    sbj2   healthy   65   F      none     None     s4        PBMC   
4     prj1    sbj3  melanoma   75   M       tr1        n     s5        PBMC   
5     prj1    sbj3  melanoma   75   M       tr1        n     s6        PBMC   
6     prj1    sbj4      lung   50   F       tr2        y     s7        PBMC   
7     prj1    sbj4      lung   50   F       tr2        y     s8       tumor   
8     prj1    sbj5   healthy   77   M      none     None     s9        PBMC   
9     prj2    sbj6   healthy   45   M      none     None    s10        PBMC   
10    prj2    sbj7   healthy   80   F      none     None    s11        PBMC   
11    prj2    sbj8  melanoma   30   F       tr1     

In [46]:
# This code will create an SQLite database with sqlalchemy
# insert the DataFrame into a table named cell_counts. 
# The if_exists='replace' argument ensures that if the table already exists, it will be replaced

# import pandas as pd
# from sqlalchemy import create_engine

# Create an SQLite database engine
# engine = create_engine('sqlite:///cell_counts_alchemy.db', echo=True)

# Convert the DataFrame to an SQLite table
# df.to_sql('cell_counts', con=engine, index=False, if_exists='replace')

# Commit changes and close the connection
# engine.dispose()

In [47]:
# To answer the question : "why to use a database and not a collection of  data frame", we list several advantages and disadvatages : 

In [48]:
print(""" 
      
A . When to Use a Database:

1 Data Persistence and Integrity :

Databases are designed for persistent storage, ensuring that data remains intact even when the application is not running. 
      
If your data involves complex relationships between entities, a relational database with multiple tables and established relationships 
is a more suitable choice. Databases enforce data integrity through constraints, ensuring accurate and consistent data.

2 Scalability:

Databases are designed to handle large volumes of data and concurrent access. 
 
3. Security:

Databases offer built-in security features, including user authentication, access control, and encryption. 
      
4. Querying and Indexing:

Databases support efficient querying and indexing, making it easy to retrieve specific subsets of data based on conditions. 

""") 


 
      
A . When to Use a Database:

1 Data Persistence and Integrity :

Databases are designed for persistent storage, ensuring that data remains intact even when the application is not running. 
      
If your data involves complex relationships between entities, a relational database with multiple tables and established relationships 
is a more suitable choice. Databases enforce data integrity through constraints, ensuring accurate and consistent data.

2 Scalability:

Databases are designed to handle large volumes of data and concurrent access. 
 
3. Security:

Databases offer built-in security features, including user authentication, access control, and encryption. 
      
4. Querying and Indexing:

Databases support efficient querying and indexing, making it easy to retrieve specific subsets of data based on conditions. 




In [49]:
print(""" 
      
B . When to Use a DataFrame (multiple DataFrames) :

1. Small to Medium-Scale Data:

For small to medium-scale datasets that can fit comfortably in memory, using a collection of DataFrames may be simple solution than setting up a database.
     
2. Prototyping and Development:

During the early stages of development, using DataFrames is simpler.
      
3. Analytical Workflows:

If our application involves extensive data analysis, and visualization, working with DataFrames in packages like Pandas is convenient. 

4. Single-User Applications:

If our application is single-user and doesn't require concurrent access to the data, using a collection of DataFrames might be sufficient.

5. Flexibility in Schema:

DataFrames are flexible in terms of schema changes. 

""")


 
      
B . When to Use a DataFrame (multiple DataFrames) :

1. Small to Medium-Scale Data:

For small to medium-scale datasets that can fit comfortably in memory, using a collection of DataFrames may be simple solution than setting up a database.
     
2. Prototyping and Development:

During the early stages of development, using DataFrames is simpler.
      
3. Analytical Workflows:

If our application involves extensive data analysis, and visualization, working with DataFrames in packages like Pandas is convenient. 

4. Single-User Applications:

If our application is single-user and doesn't require concurrent access to the data, using a collection of DataFrames might be sufficient.

5. Flexibility in Schema:

DataFrames are flexible in terms of schema changes. 




In [50]:
	# Re-writing the piece of code above

In [51]:
query_result = pd.read_sql('SELECT * FROM cell_counts', conn)
print(query_result.head(2))

  project subject condition  age sex treatment response sample sample_type  \
0    prj1    sbj1  melanoma   70   F       tr1        y     s1        PBMC   
1    prj1    sbj1  melanoma   70   F       tr1        y     s2        PBMC   

   time_from_treatment_start  b_cell  cd8_t_cell  cd4_t_cell  nk_cell  \
0                        0.0   36000       24000       42000     6000   
1                        7.0   30000       22000       40000     2000   

   monocyte  
0     12000  
1      6000  


In [52]:
# Based on the schema you provide in (1), please write a query to summarize the number of subjects available for each condition.

In [53]:
query2 = 'SELECT condition, COUNT(*) AS num_subjects FROM cell_counts GROUP BY condition;'
query2_results = pd.read_sql(query2, conn)

# Print the result
print(query2_results)

  condition  num_subjects
0   healthy             4
1      lung             4
2  melanoma             9


In [54]:
# Please write a query that returns all melanoma PBMC samples at baseline (time_from_treatment_start is 0) from patients who have treatment tr1.

In [67]:
query3 = 'SELECT * FROM cell_counts WHERE condition = "melanoma" AND sample_type = "PBMC" AND time_from_treatment_start >= 0 AND treatment = "tr1";'
str(query3)

query3_results = pd.read_sql(query3, conn)

# Print the result
print(query3_results.head(2))
print(query3_results.shape)

  project subject condition  age sex treatment response sample sample_type  \
0    prj1    sbj1  melanoma   70   F       tr1        y     s1        PBMC   
1    prj1    sbj1  melanoma   70   F       tr1        y     s2        PBMC   

   time_from_treatment_start  b_cell  cd8_t_cell  cd4_t_cell  nk_cell  \
0                        0.0   36000       24000       42000     6000   
1                        7.0   30000       22000       40000     2000   

   monocyte  
0     12000  
1      6000  
(9, 15)


In [56]:
# create a new table based on the filtering criteria listed above

In [57]:
create_table_query = "CREATE TABLE melanoma_samples_TREAT1 AS SELECT * FROM cell_counts WHERE condition = 'melanoma' AND sample_type = 'PBMC' AND time_from_treatment_start >= 0 AND treatment = 'tr1'";

In [58]:
cursor.execute(create_table_query)

In [59]:
# •	How many samples from each project 

In [60]:
query4 = 'SELECT project, COUNT(*) AS num_samples FROM melanoma_samples_TREAT1 GROUP BY project';
query4_results = pd.read_sql(query4, conn)

# Print the result
print(query4_results)

  project  num_samples
0    prj1            5
1    prj2            2
2    prj3            2


In [61]:
# •	How many responders/non-responders

In [62]:
query5 = 'SELECT response, COUNT(*) AS num_samples FROM melanoma_samples_TREAT1 GROUP BY response';
query5_results = pd.read_sql(query5, conn)
print(query5_results)

  response  num_samples
0        n            3
1        y            6


In [63]:
# another way to write the same query

In [64]:
query6 = 'SELECT response, COUNT(sample) AS sample_count FROM melanoma_samples_TREAT1 GROUP BY response';
query6_results = pd.read_sql(query6, conn)
print(query6_results)

  response  sample_count
0        n             3
1        y             6


In [65]:
# How many males, females

In [66]:
query7='SELECT sex, COUNT(*) AS gender_count FROM melanoma_samples_TREAT1 GROUP BY sex';
query7_results = pd.read_sql(query7, conn)
print(query7_results)

  sex  gender_count
0   F             5
1   M             4
